In [1]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,ChebConv, SAGEConv, GATConv, GINConv
from torch.nn import Linear

In [2]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, activationGNN, activationOut):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # Initialize the layers
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fullyConnected = Linear(hidden_channels, dataset.num_classes)
        self.activationGNN = activationGNN
        self.activationOut = activationOut

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.activationGNN(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.activationGNN(x)
        x = F.dropout(x, p=0.5, training=self.training)

        if self.activationOut == torch.softmax:
            x = self.activationOut(self.fullyConnected(x), dim=1)
        else:
            x = self.activationOut(self.fullyConnected(x))
        return x

In [4]:
model = GCN(hidden_channels=64, activationGNN=F.leaky_relu, activationOut=torch.softmax)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [5]:
# Access the weights of each layer
conv1_weights = model.conv1.lin.weight.clone()
conv2_weights = model.conv2.lin.weight.clone()

In [6]:
def train():
      model.train()
      optimizer.zero_grad()
      out = model(data.x, data.edge_index)
      # Używamy tylko nodów z labelkami do obliczenia funkcji straty
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

In [7]:
def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)
      test_correct = pred[data.test_mask] == data.y[data.test_mask]
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
      return test_acc, pred

In [8]:
# Funckja do trenowania modelu przez zadaną liczbę epok
def final_train(epochs):
    losses = []
    for epoch in range(0, epochs):
        loss = train()
        losses.append(loss)
    return losses

In [9]:
loss = final_train(1000)

In [10]:
test()

(0.783, tensor([3, 4, 4,  ..., 1, 3, 3]))

### Losowa reinicjalizacja.

In [11]:
def randomReinitalizeW1(original_tensor):
    clone = original_tensor.clone()
    for i, neuron in enumerate(original_tensor):
        for j, elem in enumerate(neuron) :
            if elem != 0: # pomijamy sprunowane wagi
                tensor = torch.empty(1)
                tensor.uniform_(-0.1, 0.1)
                clone[i][j].fill_(tensor.item())
    return clone

In [12]:
import torch.nn.utils.prune as prune

In [13]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [14]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [15]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)
model.conv2.lin.weight = randomReinitalizeW1(model.conv2.lin.weight)

In [16]:
loss = final_train(1000)
test()

(0.784, tensor([3, 4, 4,  ..., 5, 3, 3]))

In [21]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.4,  # Prune 20% of the parameters
)

In [23]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)
model.conv2.lin.weight = randomReinitalizeW1(model.conv2.lin.weight)

In [24]:
loss = final_train(1000)
test()

(0.77, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [25]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  # Prune 20% of the parameters
)

In [26]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)
model.conv2.lin.weight = randomReinitalizeW1(model.conv2.lin.weight)

In [27]:
loss = final_train(1000)
test()

(0.784, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [29]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.8,  # Prune 20% of the parameters
)

In [30]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)
model.conv2.lin.weight = randomReinitalizeW1(model.conv2.lin.weight)

In [31]:
loss = final_train(1000)
test()

(0.778, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [32]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.99,  # Prune 20% of the parameters
)

In [34]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)
model.conv2.lin.weight = randomReinitalizeW1(model.conv2.lin.weight)

In [35]:
loss = final_train(1000)
test()

(0.714, tensor([3, 4, 4,  ..., 3, 0, 0]))

In [36]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.99,  # Prune 20% of the parameters
)

In [38]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)
model.conv2.lin.weight = randomReinitalizeW1(model.conv2.lin.weight)

In [39]:
loss = final_train(1000)
test()

(0.319, tensor([3, 3, 3,  ..., 3, 3, 3]))

In [ ]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [14]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.99,  # Prune 20% of the parameters
)

In [15]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 99.00%


In [16]:
loss = final_train(1000)

In [17]:
test()

(0.361, tensor([3, 6, 6,  ..., 1, 3, 3]))

In [17]:
def recoverOriginalWeightsW1():
    clone = model.conv1.lin.weight.clone()
    for i, neuron in enumerate(model.conv1.lin.weight):
        for j, elem in enumerate(neuron) :
#             print(clone[i][j])
#             print(conv1_weights[i][j])
            if elem != 0: # pomijamy sprunowane wagi
                clone[i][j].fill_(conv1_weights[i][j].detach())
    return clone

In [18]:
def recoverOriginalWeightsW2():
    clone = model.conv2.lin.weight.clone()
    for i, neuron in enumerate(model.conv2.lin.weight):
        for j, elem in enumerate(neuron) :
            if elem != 0: # pomijamy sprunowane wagi
                clone[i][j].fill_(conv2_weights[i][j].detach())
    return clone

In [19]:
model.conv1.lin.weight = recoverOriginalWeightsW1()

In [20]:
model.conv2.lin.weight = recoverOriginalWeightsW2()

In [21]:
loss = final_train(1000)

In [22]:
test()

(0.781, tensor([3, 4, 4,  ..., 5, 3, 3]))

In [23]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.4,  # Prune 20% of the parameters
)

In [24]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 52.00%


In [25]:
model.conv1.lin.weight = recoverOriginalWeightsW1()
model.conv2.lin.weight = recoverOriginalWeightsW2()

In [26]:
loss = final_train(1000)

In [27]:
test()

(0.778, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [28]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  # Prune 20% of the parameters
)

In [30]:
model.conv1.lin.weight = recoverOriginalWeightsW1()
model.conv2.lin.weight = recoverOriginalWeightsW2()

In [31]:
loss = final_train(1000)

In [32]:
test()

(0.783, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [33]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  # Prune 20% of the parameters
)

In [35]:
model.conv1.lin.weight = recoverOriginalWeightsW1()
model.conv2.lin.weight = recoverOriginalWeightsW2()

In [36]:
loss = final_train(1000)
test()

(0.787, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [37]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.95,  # Prune 20% of the parameters
)

In [38]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 99.62%


In [39]:
model.conv1.lin.weight = recoverOriginalWeightsW1()
model.conv2.lin.weight = recoverOriginalWeightsW2()

In [40]:
loss = final_train(1000)
test()

(0.684, tensor([3, 4, 4,  ..., 1, 3, 3]))